# Gliderpy

gliderpy builds off of the functionality of erddapy to make glider data access, manipulation, and visualization even simpler 

(https://github.com/oceanhackweek/ohw20-proj-glide-data-fetcher) 

In [ ]:
from pathlib import Path
import sys
notebook_dir = !pwd
sys.path.append(str( Path(notebook_dir[0]).parent.absolute()))

# Here is an example of plain erddapy

Copied from the [errdapy intro](https://ioos.github.io/erddapy/quick_intro-output.html#Longer-introduction)

In [ ]:
from erddapy import ERDDAP


e = ERDDAP(
    server="https://gliders.ioos.us/erddap",
    protocol="tabledap",
    response="csv",
)
e.dataset_id = "whoi_406-20160902T1700"

e.variables = [
    "depth",
    "latitude",
    "longitude",
    "salinity",
    "temperature",
    "time",
]

df = e.to_pandas(
    index_col="time (UTC)",
    parse_dates=True,
)
df.head()

# Fetch glider data using glider ID

Erddapy is great but we want to try to make it even more simple with fewer lines of code with our **gliderpy** package.

This emulates the UI of [argopy](https://github.com/euroargodev/argopy)

In [ ]:
from gliderpy.fetchers import GliderDataFetcher

glider_grab = GliderDataFetcher()

glider_grab.fetcher.dataset_id = "whoi_406-20160902T1700"
df = glider_grab.to_pandas()
df.head()

Note how we attain the same output as erddapy with fewer lines and a cleaner UI

Try with a different dataset

In [ ]:
glider_grab = GliderDataFetcher()
glider_grab.fetcher.dataset_id = "sp022-20170209T1616"

df = glider_grab.to_pandas()
df.head()

# Now we will pass time and space constraints

The user can supply a bounding box of lat and lon as well as start and end times in arbitrary format to constrain their search

In [ ]:
glider_grab = GliderDataFetcher()
glider_grab.fetcher.dataset_id = "whoi_406-20160902T1700"
glider_grab.query(38, 41, -72, -69, "2016-09-20T01:00:00", "25th sept 2016")

df = glider_grab.to_pandas()
df.head()

### Find out what datasets are avaialabe on a server

Calling get_ids from the class searches for all databases which have "glider in their metadata"

In [ ]:
from gliderpy.fetchers import DatasetList

datasets = DatasetList()
ds_ids = datasets.get_ids()

print(f'found {len(ds_ids)} datasets matching the search terms {datasets.search_terms} on the server {datasets.e.server}')
ds_ids

The user can supply their own list of search terms. Let's look for datasets from UW

In [ ]:
datasets = DatasetList()
datasets.search_terms = ["UW"]
uw_glider_ids = datasets.get_ids()

print(f'found {len(uw_glider_ids)} datasets matching the search terms {datasets.search_terms} on the server {datasets.e.server}')

uw_glider_ids

Finally, a combined search for datasets from UW or Scripps

In [ ]:
datasets = DatasetList()
datasets.search_terms = ["UW", "Scripps"]
combi_ids = datasets.get_ids()

print(f'found {len(combi_ids)} datasets matching the search terms {datasets.search_terms} on the server {datasets.e.server}')

combi_ids

We'll take an id from the list and pass it back into our data fetcher

In [ ]:
uw_grab = GliderDataFetcher()
uw_grab.fetcher.dataset_id = uw_glider_ids[18]

df = uw_grab.to_pandas()
df.head()

## Let's plot the track map for a dataset

In [ ]:
#Colored by temperature--will add kwargs in a future update
plot_track(df)

## Now the transect for a user defined variable in that dataset

In [ ]:
#Variable options: salinity (1) or temperature (Celcius)
plot_transect(df,"temperature (Celcius)")

In [ ]:
plot_transect(df,"salinity (1)")